In [2]:
import numpy as np
import pandas as pd

In [14]:
titanic_df = pd.read_csv("C:/Users/Umang/Downloads/titanic.csv")
titanic_df.head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
5,1,1,"Anderson, Mr. Harry",male,48.0000,0,0,19952,26.5500,E12,S,3,NaN,"New York, NY"
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,13502,77.9583,D7,S,10,NaN,"Hudson, NY"
7,1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,112050,0.0000,A36,S,NaN,NaN,"Belfast, NI"
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY"
9,1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,PC 17609,49.5042,NaN,C,NaN,22.0,"Montevideo, Uruguay"


In [15]:
titanic_df.shape

(1309, 14)

Estimating a LPM model:

In [105]:
import statsmodels.formula.api as smf

# creating a fitted model 
lm = smf.ols(formula='survived ~ C(pclass) + sex', data=titanic_df).fit()
# print the coefficients
lm.params
# print summary
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               survived   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.340
Method:                 Least Squares   F-statistic:                     225.5
Date:                Wed, 11 Apr 2018   Prob (F-statistic):          7.79e-118
Time:                        23:07:47   Log-Likelihood:                -639.22
No. Observations:                1309   AIC:                             1286.
Df Residuals:                    1305   BIC:                             1307.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.8990      0.025     35.398      0.000       0.849       0.949
C(pclass)[T.2]    -0.1577      0.032     -4.872      0.000      -0.221      -0.094
C(pclass)[T.3]    -0.2926      0.027    -10.957      0.000      -0.345      -0.240
sex[T.male]       -0.5049      0.023    -21.974      0.000      -0.550      -0.460
==============================================================================
Omnibus:                       60.055   Durbin-Watson:                   1.760
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.798
Skew:                           0.557   Prob(JB):                     1.90e-15
Kurtosis:                       2.961   Cond. No.                         5.25
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

For Prediction: 

In [106]:
x_new = lm.predict()
print(x_new.max())
print(x_new.min())

0.8989839322447655
0.10147811470358392


In [107]:
X_new = pd.DataFrame({'SURVIVAL': [titanic_df.survived.min(), titanic_df.survived.max()]})
X_new.head()

,SURVIVAL
0,0
1,1


These are exactly equal to zero or one. 

In [108]:
survived = np.sum(x_new > 0.5)
y_pred = survived/len(titanic_df)*100
print("Predicted percentage survived: ",y_pred)

Predicted percentage survived:  35.59969442322384


In [109]:
survivedtrue = sum(titanic_df[titanic_df.survived==1].survived)
y_true = (survivedtrue)/len(titanic_df)*100
print("Actual percentage survived: ",y_true)

Actual percentage survived:  38.19709702062643


In [110]:
#np.mean(titanic_df.survived == (x_new > 0.5)+0)
from sklearn.metrics import accuracy_score
accuracy_score(titanic_df.survived , (x_new > 0.5)+0)

0.7799847211611918

Estimate a similar LPM, but now allow the passenger class effect to differ between men and women 

In [111]:
# creating a similar fitted model 
lm2 = smf.ols(formula='survived ~ C(pclass) * sex', data=titanic_df).fit()
# printing the coefficients
lm2.params
# printing summary
lm2.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               survived   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     153.2
Date:                Wed, 11 Apr 2018   Prob (F-statistic):          4.02e-128
Time:                        23:08:13   Log-Likelihood:                -609.86
No. Observations:                1309   AIC:                             1232.
Df Residuals:                    1303   BIC:                             1263.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.9653      0.032     29.974      0.000       0.902       1.028
C(pclass)[T.2]                -0.0785      0.049     -1.587      0.113      -0.176       0.019
C(pclass)[T.3]                -0.4745      0.042    -11.414      0.000      -0.556      -0.393
sex[T.male]                   -0.6245      0.043    -14.436      0.000      -0.709      -0.540
C(pclass)[T.2]:sex[T.male]    -0.1161      0.064     -1.801      0.072      -0.243       0.010
C(pclass)[T.3]:sex[T.male]     0.2859      0.054      5.340      0.000       0.181       0.391
==============================================================================
Omnibus:                      113.251   Durbin-Watson:                   1.746
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              142.526
Skew:                           0.805   Prob(JB):                     1.12e-31
Kurtosis:                       3.136   Cond. No.                         13.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [102]:
x_new2 = lm2.predict()
print(x_new2.max())
print(x_new2.min())

0.9652777777777743
0.14619883040935766


In [112]:
#Accuracy
accuracy_score(titanic_df.survived , (x_new2 > 0.5)+0)

0.7830404889228418

Interpretation: The Class1 and Class2 intercepts are negative, and thus women from higher class had higher chances of survival. Also, men had lower chances of survival in comparision to women. 